可以在训练期间和之后保存模型进度。这意味着模型可以从中断的地方继续进行，避免了长时间的训练。保存还意味着您可以共享模型，而其他人可以重新创建您的作品。在发布研究模型和技术时，大多数机器学习从业者分享：

* 创建模型的代码，以及
* 模型的训练权重或参数

共享这些数据可帮助其他人了解模型的工作原理，并使用新数据自己尝试。

In [2]:
# Required to save models in HDF5 format
# !pip install -q pyyaml h5py  

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.0.0


获取演示数据

In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

定义简单模型

In [5]:
# Define a simple sequential model
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## 一、训练过程中保存权重参数

定义保存路径，以及在训练期间执行保存训练参数

In [6]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 640/1000 [==================>...........] - ETA: 0s - loss: 1.4740 - accuracy: 0.5781     
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 1ms/sample - loss: 1.1959 - accuracy: 0.6690 - val_loss: 0.7248 - val_accuracy: 0.7790
Epoch 2/10
 640/1000 [==================>...........] - ETA: 0s - loss: 0.4633 - accuracy: 0.8734
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 201us/sample - loss: 0.4372 - accuracy: 0.8820 - val_loss: 0.5440 - val_accuracy: 0.8350
Epoch 3/10
 640/1000 [==================>...........] - ETA: 0s - loss: 0.2826 - accuracy: 0.9250
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 201us/sample - loss: 0.2889 - accuracy: 0.9260 - val_loss: 0.4745 - val_accuracy: 0.8480
Epoch 4/10
 608/1000 [=================>............] - ETA: 0s - loss: 0.2115 - accuracy: 0.9539
Ep

In [8]:
!dir {checkpoint_dir}

 驱动器 C 中的卷是 系统
 卷的序列号是 0864-E2D5

 C:\Users\30660\Documents\Project\python\jupyter\TensorFlow2.0官方\training_1 的目录

2019/10/14  19:18    <DIR>          .
2019/10/14  19:18    <DIR>          ..
2019/10/14  19:18                71 checkpoint
2019/10/14  19:18             2,064 cp.ckpt.data-00000-of-00002
2019/10/14  19:18         4,884,624 cp.ckpt.data-00001-of-00002
2019/10/14  19:18             1,251 cp.ckpt.index
               4 个文件      4,888,010 字节
               2 个目录 819,545,059,328 可用字节


未经训练的模型

In [9]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 2.3595 - accuracy: 0.0560
Untrained model, accuracy:  5.60%


加载模型参数后

In [10]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4233 - accuracy: 0.8710
Restored model, accuracy: 87.10%


## 二、设定保存频率

每五个时期保存一次，并且按照当前训练轮数命名保存

In [11]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
              train_labels,
              epochs=50, 
              callbacks=[cp_callback],
              validation_data=(test_images,test_labels),
              verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [12]:
!dir {checkpoint_dir}

 驱动器 C 中的卷是 系统
 卷的序列号是 0864-E2D5

 C:\Users\30660\Documents\Project\python\jupyter\TensorFlow2.0官方\training_2 的目录

2019/10/14  19:19    <DIR>          .
2019/10/14  19:19    <DIR>          ..
2019/10/14  19:19                81 checkpoint
2019/10/14  19:19               541 cp-0000.ckpt.data-00000-of-00002
2019/10/14  19:19         1,628,200 cp-0000.ckpt.data-00001-of-00002
2019/10/14  19:19               406 cp-0000.ckpt.index
2019/10/14  19:19             2,076 cp-0005.ckpt.data-00000-of-00002
2019/10/14  19:19         4,884,624 cp-0005.ckpt.data-00001-of-00002
2019/10/14  19:19             1,251 cp-0005.ckpt.index
2019/10/14  19:19             2,076 cp-0010.ckpt.data-00000-of-00002
2019/10/14  19:19         4,884,624 cp-0010.ckpt.data-00001-of-00002
2019/10/14  19:19             1,251 cp-0010.ckpt.index
2019/10/14  19:19             2,076 cp-0015.ckpt.data-00000-of-00002
2019/10/14  19:19         4,884,624 cp-0015.ckpt.data-00001-of-00002
2019/10/14  19:19             1,251 cp-0015.

In [13]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

重置模型，并进行测试

In [14]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.5096 - accuracy: 0.8750
Restored model, accuracy: 87.50%


## 三、手动保存参数

手动保存参数

In [15]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.5096 - accuracy: 0.8750
Restored model, accuracy: 87.50%


## 四、保存整个模型

保存整个模型

In [16]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model shuold be saved to HDF5.
model.save('my_model.h5') 

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 384us/sample - loss: 1.1750 - accuracy: 0.6770
Epoch 2/5
1000/1000 [==============================] - 0s 94us/sample - loss: 0.4168 - accuracy: 0.8860
Epoch 3/5
1000/1000 [==============================] - 0s 96us/sample - loss: 0.2785 - accuracy: 0.9200
Epoch 4/5
1000/1000 [==============================] - 0s 93us/sample - loss: 0.2001 - accuracy: 0.9510
Epoch 5/5
1000/1000 [==============================] - 0s 95us/sample - loss: 0.1568 - accuracy: 0.9670


从保存的模型开始创建

In [17]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


检查性能

In [18]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1 - 0s - loss: 0.4943 - accuracy: 0.8570
Restored model, accuracy: 85.70%


上面这种保存方式，可以保存一切，包括权重值，模型结构，和优化器配置

## 五、不使用h5格式的文件保存

SavedModel格式是一个包含protobuf二进制文件和Tensorflow检查点的目录。检查保存的模型目录：

In [20]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
model.save('saved_model/my_model') 

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 460us/sample - loss: 1.1521 - accuracy: 0.6770
Epoch 2/5
1000/1000 [==============================] - 0s 96us/sample - loss: 0.4343 - accuracy: 0.8800
Epoch 3/5
1000/1000 [==============================] - 0s 95us/sample - loss: 0.2965 - accuracy: 0.9150
Epoch 4/5
1000/1000 [==============================] - 0s 94us/sample - loss: 0.2209 - accuracy: 0.9480
Epoch 5/5
1000/1000 [==============================] - 0s 95us/sample - loss: 0.1600 - accuracy: 0.9620
INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [23]:
# my_model directory
!dir saved_model

 驱动器 C 中的卷是 系统
 卷的序列号是 0864-E2D5

 C:\Users\30660\Documents\Project\python\jupyter\TensorFlow2.0官方\saved_model 的目录

2019/10/14  19:22    <DIR>          .
2019/10/14  19:22    <DIR>          ..
2019/10/14  19:22    <DIR>          .ipynb_checkpoints
2019/10/14  19:22    <DIR>          my_model
               0 个文件              0 字节
               4 个目录 823,388,827,648 可用字节


In [24]:
# Contains an assets folder, saved_model.pb, and variables folder.
!dir saved_model\my_model

 驱动器 C 中的卷是 系统
 卷的序列号是 0864-E2D5

 C:\Users\30660\Documents\Project\python\jupyter\TensorFlow2.0官方\saved_model\my_model 的目录

2019/10/14  19:22    <DIR>          .
2019/10/14  19:22    <DIR>          ..
2019/10/14  19:22    <DIR>          assets
2019/10/14  19:22            81,456 saved_model.pb
2019/10/14  19:22    <DIR>          variables
               1 个文件         81,456 字节
               4 个目录 823,387,885,568 可用字节


加载模型

In [25]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


进行评估

In [26]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

1000/1 - 0s - loss: 0.4855 - accuracy: 0.8500
Restored model, accuracy: 85.00%
(1000, 10)


保存自定义对象
如果使用的是SavedModel格式，则可以跳过此部分。HDF5和SavedModel之间的主要区别在于，HDF5使用对象配置保存模型体系结构，而SavedModel保存执行图。因此，SavedModels能够保存自定义对象，例如子类化模型和自定义层，而无需原始代码。

要将自定义对象保存到HDF5，必须执行以下操作：

1. get_config在您的对象中定义一个方法，以及一个可选的from_config类方法。
    * get_config(self) 返回重新创建对象所需的参数的JSON可序列化字典。
    * from_config(cls, config)使用返回的配置从get_config创建新对象。默认情况下，此函数将配置用作初始化kwargs（return cls(**config)）。
2. custom_objects加载模型时，将对象传递给参数。参数必须是将字符串类名称映射到Python类的字典。例如tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})